In [1]:
# This code is based on the code provided by Januka Attanayake from UoM. The data logger response is included by modifying 
# the "StageGain" value in the generated xml file by this code for all channels: 
# "StageGain = sensitivity * counts per volts (cpv)" 

## Use the following code to check the correct format for station keys:

# import obspy
# from obspy.core.inventory import Inventory, Network, Station, Channel, Site
# from obspy.clients.nrl import NRL
# nrl = NRL()
# print(nrl.sensors) 
# print(nrl.sensors['Nanometrics']['Trillium Compact 120 (Vault, Posthole, OBS)']['1500 V/m/s'])   


In [2]:
###############################################################
#                       USER INPUT                            #
###############################################################
stnm       = 'WPSH'                       # station code                       
ntwk       = 'AB'                         # network code, e.g. AB, OZ
locode     = '60'                         # location code, e.g. 60, B0, 00
creator    = 'NRL'                        # creater  
#-----------------
chan_Z     = 'DHZ'                        
chan_E     = 'DHE'
chan_N     = 'DHN'

# chan_Z     = 'DHZ'                        
# chan_E     = 'DHE'
# chan_N     = 'DHN'
#-----------------
stla       = -39.097729                 # station latitude (degree decimal) e.g. -65.9876
stlo       = 146.381561               # station longitude (degree decimal) e.g. 130.7654
stel       = 276.000           # station elevation above MSL (meters) e.g. 354
stdp       = 0                            # depth of the instrument from the surface (meters) e.g. 10
Fs         = 250                          # sampling rate (i.e. 1/dt)  e.g. if dt = 0.025s, then Fs = 40
stdate     = '2021,1,1,0,0,0.00'          # start data + time of the instrument = ('yyyy,mm,dd,hh,mn,sec.decimal') < event start time
endate     = '2599,1,2,0,0,0.00'          # end date + time of the instrument   = ('yyyy,mm,dd,hh,mn,sec.decimal') > event stop time
sitenm     = 'WPSH'                       # site name e.g. Gippsland
stcomm     = 'AB SRC/UoM Netwrok'         # comments about the station e.g. At the top of a hill
#-----------------

# sensornm   = 'Nanometrics'                  # name of the sensor e.g. Streckeisen or Guralp
# sensortype = 'Trillium Compact 120 (Vault, Posthole, OBS)'                  # type of the sensor eg. STS-1
# sensorT    = '1500 V/m/s'             # natural period of the sensor e.g. 360 seconds
# senvty     = ''                     # some sensors have sensitivity as a variable
# sensorkey  = 3                          # number of strings required to uniquely identify a response for an instrument


# sensornm   = 'Guralp'                   # name of the sensor e.g. Streckeisen or Guralp
# sensortype = 'CMG-40T'                  # type of the sensor eg. STS-1
# sensorT    = '10s - 100Hz'              # natural period of the sensor e.g. 360 seconds
# senvty     = '1600'                     # some sensors have sensitivity as a variable
# sensorkey  = 4                          # number of strings required to uniquely identify a response for an instrument
                
sensornm   = 'Guralp'
sensortype = 'CMG-6T'
sensorT    = '1s - 100Hz'
senvty     = '2400'
sensorkey  = 4

# sensornm   = 'Sprengnether (now Eentec)'
# sensortype = 'S6000'
# sensorT    = '10000 and 6470 Ohms'
# senvty     = '110 V/m/s'
# sensorkey  = 3

# sensornm   = 'Guralp'
# sensortype = 'CMG-5TC'
# sensorT    = '2G_20V'
# senvty     = '100 Hz'
# sensorkey  = 4

In [3]:
###############################################################
#             START BUILDING THE STATIONXML FILE              #
###############################################################
import obspy
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
from obspy.clients.nrl import NRL
from obspy import read

inv   = Inventory(networks=[], source=creator)
net   = Network(code=ntwk, stations=[], description=stcomm, start_date=obspy.UTCDateTime(stdate))
sta   = Station(code=stnm, latitude=stla, longitude=stlo,elevation=stel, creation_date=obspy.UTCDateTime(stdate),
              site=Site(name=sitenm))

chanZ = Channel(code=chan_Z, location_code=locode, latitude=stla, longitude=stlo, elevation=stel, depth=stdp,
              azimuth=0, dip=-90, sample_rate=Fs)
chanN = Channel(code=chan_N, location_code=locode, latitude=stla, longitude=stlo, elevation=stel, depth=stdp,
              azimuth=0, dip=0, sample_rate=Fs)
chanE = Channel(code=chan_E, location_code=locode, latitude=stla, longitude=stlo, elevation=stel, depth=stdp,
              azimuth=90, dip=0, sample_rate=Fs)


###############################################################
#      QUERY NRL LIBRARY AND GET RESPONSE INFORMATION         #
###############################################################
nrl = NRL()

if sensorkey == 4:
    response = nrl.get_sensor_response(sensor_keys=[sensornm, sensortype, sensorT, senvty])
    
elif sensorkey == 3:
    response = nrl.get_sensor_response(sensor_keys=[sensornm, sensortype,sensorT])
    
else:
    response = nrl.get_sensor_response(sensor_keys=[sensornm, sensortype])

###############################################################
#    PUT ALL PIECES TOGETHER AND WRITE OUT THE STATIONXML     #
###############################################################   
chanZ.response = response
sta.channels.append(chanZ)
net.stations.append(sta)
inv.networks.append(net)

chanN.response = response
sta.channels.append(chanN)

chanE.response = response
sta.channels.append(chanE)

outfile = (stnm + '.xml')
inv.write(outfile, format="stationxml", validate=True)
